Package Imports

In [1]:
#Muhammad Sa'ood Shah 221021204
#Yruvan Ramjan 221004686
import numpy as np
import keras
import os
from pathlib import Path
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Bidirectional, LSTM,Embedding,GRU,Dense,Concatenate
from keras.models import Model
from keras.layers import Input
from keras.models import load_model
import numpy as np
import pandas as pd
import nltk
from nltk.translate.bleu_score import corpus_bleu
from collections import defaultdict

Load Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_path='/content/drive/MyDrive/Colab Notebooks/fra.txt'

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

Mounted at /content/drive


Pre Processing

In [3]:
from sklearn.model_selection import train_test_split
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:

            target_characters.add(char)
for i in range(10):
    print(f"Input {i+1}: {input_texts[i]}")
    print(f"Target {i+1}: {target_texts[i]}")
    print("-" * 50)
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
print(input_texts[0])
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
input_train, input_test, target_train, target_test = train_test_split(input_texts, target_texts, test_size=0.2, random_state=42)
encoder_input_data = np.zeros(
    (len(input_train), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_train), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_train), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_train, target_train) in enumerate(zip(input_train, target_train)):
    for t, char in enumerate(input_train):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_train):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Input 1: Go.
Target 1: 	Va !

--------------------------------------------------
Input 2: Go.
Target 2: 	Marche.

--------------------------------------------------
Input 3: Go.
Target 3: 	En route !

--------------------------------------------------
Input 4: Go.
Target 4: 	Bouge !

--------------------------------------------------
Input 5: Hi.
Target 5: 	Salut !

--------------------------------------------------
Input 6: Hi.
Target 6: 	Salut.

--------------------------------------------------
Input 7: Run!
Target 7: 	Cours !

--------------------------------------------------
Input 8: Run!
Target 8: 	Courez !

--------------------------------------------------
Input 9: Run!
Target 9: 	Prenez vos jambes à vos cous !

--------------------------------------------------
Input 10: Run!
Target 10: 	File !

--------------------------------------------------
Go.
Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14

In [4]:
def vectorize_texts(texts, token_index, max_seq_length, num_tokens):
    data = np.zeros((len(texts), max_seq_length, num_tokens), dtype="float32")
    for i, text in enumerate(texts):
        for t, char in enumerate(text):
            data[i, t, token_index[char]] = 1.0
        # Padding with space character
        data[i, t + 1:, token_index[" "]] = 1.0
    return data

# Vectorize the test data
encoder_input_data_test = vectorize_texts(input_test, input_token_index, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data_test = vectorize_texts(target_test, target_token_index, max_decoder_seq_length, num_decoder_tokens)


Encoder-decoder model with bidirectional LSTM encoder and an LSTM decoder

In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs,forward_h,forward_c ,backward_h, backward_c = encoder(encoder_inputs)

state_h=Concatenate()([forward_h,backward_h])
state_c=Concatenate()([forward_c,backward_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c] #final state of encoder
# Captures semantic info of input seq and represents it as a context Vector
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_latent_dim = latent_dim * 2
decoder_lstm = LSTM(decoder_latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)#Output of previous of decoder at prev timetstamp and encoder states as input
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[es],
    validation_split=0.2,
)
# Save model
model.save("/content/drive/MyDrive/Colab Notebooks/BiDirectional_lstm_Encoder_lstm_decoderV3.keras")

Epoch 1/100
100/100 [==============================] - 11s 67ms/step - loss: 1.1681 - accuracy: 0.7263 - val_loss: 0.8905 - val_accuracy: 0.7447
Epoch 2/100
100/100 [==============================] - 5s 53ms/step - loss: 1.0813 - accuracy: 0.7478 - val_loss: 0.9084 - val_accuracy: 0.7545
Epoch 3/100
100/100 [==============================] - 6s 56ms/step - loss: 0.8170 - accuracy: 0.7768 - val_loss: 0.7562 - val_accuracy: 0.7911
Epoch 4/100
100/100 [==============================] - 6s 57ms/step - loss: 0.7187 - accuracy: 0.8002 - val_loss: 0.6715 - val_accuracy: 0.8100
Epoch 5/100
100/100 [==============================] - 6s 56ms/step - loss: 0.6472 - accuracy: 0.8140 - val_loss: 0.6193 - val_accuracy: 0.8217
Epoch 6/100
100/100 [==============================] - 6s 55ms/step - loss: 0.6011 - accuracy: 0.8258 - val_loss: 0.5818 - val_accuracy: 0.8311
Epoch 7/100
100/100 [==============================] - 5s 54ms/step - loss: 0.5678 - accuracy: 0.8347 - val_loss: 0.5530 - val_accuracy

Encoder-decoder model with bidirectional GRU encoder and a GRU decoder

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(GRU(latent_dim, return_state=True))
encoder_outputs,forward_h,backward_h = encoder(encoder_inputs)

state_h=Concatenate()([forward_h,backward_h])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_latent_dim = latent_dim * 2
decoder_gru = GRU(decoder_latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=state_h)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model2 = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
model2.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model2.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[es],
    validation_split=0.2,
)
# Save model
model2.save("/content/drive/MyDrive/Colab Notebooks/BiDirectional_gru_Encoder_gru_decoderV2.keras")

100/100 [==============================] - 8s 36ms/step - loss: 1.2896 - accuracy: 0.7305 - val_loss: 0.8603 - val_accuracy: 0.7593
Epoch 2/100
100/100 [==============================] - 2s 20ms/step - loss: 0.7505 - accuracy: 0.7901 - val_loss: 0.6530 - val_accuracy: 0.8119
Epoch 3/100
100/100 [==============================] - 2s 18ms/step - loss: 0.6073 - accuracy: 0.8254 - val_loss: 0.5727 - val_accuracy: 0.8324
Epoch 4/100
100/100 [==============================] - 2s 18ms/step - loss: 0.5464 - accuracy: 0.8416 - val_loss: 0.5268 - val_accuracy: 0.8448
Epoch 5/100
100/100 [==============================] - 2s 18ms/step - loss: 0.5106 - accuracy: 0.8505 - val_loss: 0.4977 - val_accuracy: 0.8543
Epoch 6/100
100/100 [==============================] - 2s 19ms/step - loss: 0.4846 - accuracy: 0.8579 - val_loss: 0.4783 - val_accuracy: 0.8595
Epoch 7/100
100/100 [==============================] - 2s 19ms/step - loss: 0.4636 - accuracy: 0.8634 - val_loss: 0.4593 - val_accuracy: 0.8651
Epoc

Double Layer GRU Encoder and GRU Decoder

In [ ]:
latent_dim = 256  # Latent dimensionality of the encoding space

# Define input sequence shape
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_first_layer = GRU(latent_dim, return_sequences=True, return_state=True)
encoder_outputs_first_layer, state_h1 = encoder_first_layer(encoder_inputs)

# Adding the second GRU layer
encoder_second_layer = GRU(latent_dim, return_state=True)
encoder_outputs_second_layer, state_h2 = encoder_second_layer(encoder_outputs_first_layer)

# Define the decoder with the same latent dimension
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=[state_h2])

# Dense layer to generate output tokens
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data

model3 = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Compile the model
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
model3.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model3.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[es],
    validation_split=0.2,
)
# Save model
model3.save("/content/drive/MyDrive/Colab Notebooks/doubleLayer_gru_Encoder_gru_decoderV2.keras")

Epoch 1/100
100/100 [==============================] - 7s 29ms/step - loss: 1.4295 - accuracy: 0.7182 - val_loss: 0.9362 - val_accuracy: 0.7393
Epoch 2/100
100/100 [==============================] - 2s 16ms/step - loss: 0.8332 - accuracy: 0.7709 - val_loss: 0.7397 - val_accuracy: 0.7953
Epoch 3/100
100/100 [==============================] - 1s 12ms/step - loss: 0.6882 - accuracy: 0.8057 - val_loss: 0.6403 - val_accuracy: 0.8126
Epoch 4/100
100/100 [==============================] - 1s 11ms/step - loss: 0.6178 - accuracy: 0.8206 - val_loss: 0.5918 - val_accuracy: 0.8272
Epoch 5/100
100/100 [==============================] - 1s 11ms/step - loss: 0.5800 - accuracy: 0.8298 - val_loss: 0.5640 - val_accuracy: 0.8343
Epoch 6/100
100/100 [==============================] - 1s 11ms/step - loss: 0.5518 - accuracy: 0.8373 - val_loss: 0.5419 - val_accuracy: 0.8393
Epoch 7/100
100/100 [==============================] - 1s 11ms/step - loss: 0.5310 - accuracy: 0.8427 - val_loss: 0.5213 - val_accuracy:

Inferenceing and decoding on Bi LSTM Encoder and Lstm decoder



In [21]:
from keras.models import Model
from keras.layers import Input
model_1 = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/BiDirectional_lstm_Encoder_lstm_decoderV3.keras")

# Print layer names to identify required layers
layer_names = [layer.name for layer in model_1.layers]
print(layer_names)

# Rebuild Encoder
encoder_inputs = model_1.input[0]  # input_1
encoder_outputs, forward_h, forward_c, backward_h, backward_c = model_1.layers[1].output  # lstm_1 (Bidirectional LSTM)
state_h_enc = keras.layers.Concatenate()([forward_h, backward_h])  # Concatenate forward and backward states
state_c_enc = keras.layers.Concatenate()([forward_c, backward_c])  # Concatenate forward and backward states
encoder_states = [state_h_enc, state_c_enc,encoder_outputs]
encoder_model = Model(encoder_inputs,encoder_states)
#Bi_GRU_encoder_model = Model(Bi_GRU_encoder_inputs, [Bi_GRU_encoder_outputs] + Bi_GRU_encoder_states)
# Rebuild Decoder
latent_dim = state_h_enc.shape[-1]  # Get the latent dimension size from the concatenated states
decoder_inputs = model_1.input[1]  # input_2

decoder_state_input_h = keras.Input(shape=(latent_dim ,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model_1.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model_1.layers[-1]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Testing the rebuilt models (Optional)
print("Encoder model summary:")
encoder_model.summary()
print("Decoder model summary:")
decoder_model.summary()

['input_29', 'bidirectional_2', 'input_30', 'concatenate_18', 'concatenate_19', 'lstm_5', 'dense_3']
Encoder model summary:
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_29 (InputLayer)       [(None, None, 70)]           0         []                            
                                                                                                  
 bidirectional_2 (Bidirecti  [(None, 1024),               2387968   ['input_29[0][0]']            
 onal)                        (None, 512),                                                        
                              (None, 512),                                                        
                              (None, 512),                                                        
                              (None, 512)]                         

In [23]:

import numpy as np
import pandas as pd
import nltk
from nltk.translate.bleu_score import corpus_bleu
from collections import defaultdict
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    #states_value=encoder_model.predict(input_seq, verbose=0)
    encoder_output, state_h, state_c = encoder_model.predict(input_seq, verbose=0)

    # Prepare initial states for the decoder.
    states_value = [state_h, state_c]

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence



decoded_sentences=[]


smooth = SmoothingFunction().method4
bleu_score=0
for seq_index in range(20):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences.append(decoded_sentence)
    print(f'Input text: {input_test[seq_index]}')
    print(f'Target text: {target_test[seq_index]}')
    print(f'Decoded text: {decoded_sentence}')
    reference = target_test[seq_index].strip().split()  # Split target text into tokens
    candidate = decoded_sentence.split()  # Split decoded text into tokens
    bleu_score += sentence_bleu(reference, candidate, smoothing_function=smooth)  # Calculate BLEU score with equal weights for all n-grams



print("Overall BLEU Score: " + str(bleu_score/20))



Input text: Stop yelling!
Target text: 	Cessez de hurler !

Decoded text: e portir !

Input text: Anybody hurt?
Target text: 	Quiconque a-t-il été blessé ?

Decoded text: is  s                                                       
Input text: All aboard!
Target text: 	Tout le monde à bord !

Decoded text: ill ma gagnes !

Input text: Are you lame?
Target text: 	Êtes-vous boiteux ?

Decoded text: i n  s                                                      
Input text: Who are you?
Target text: 	T'es qui toi ?

Decoded text: is s ma gagné ?

Input text: They came in.
Target text: 	Ils sont entrés.

Decoded text: is s min chante !

Input text: Get lost.
Target text: 	Décampe !

Decoded text: is le von  l                                                
Input text: I don't date.
Target text: 	Je ne sors pas avec des garçons.

Decoded text: is s ma gagné !

Input text: They love me.
Target text: 	Elles m'aiment.

Decoded text: is pour !

Input text: Sign up.
Target text: 	Inscris-toi.

Deco

Inferencing with Model 2 : Bi GRU Encoder and GRU decoder

In [26]:
from keras.models import Model
from keras.layers import Input
model2 = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/BiDirectional_gru_Encoder_gru_decoderV2.keras")

# Print layer names to identify required layers
layer_names = [layer.name for layer in model2.layers]
print(layer_names)

# Rebuild Encoder
Bi_GRU_encoder_inputs = model2.input[0]  # input_1
Bi_GRU_encoder_outputs, forward_h, backward_h = model2.layers[1].output  #  (Bidirectional GRU)
state_h_enc = keras.layers.Concatenate()([forward_h, backward_h])  # Concatenate forward and backward states

Bi_GRU_encoder_states = [state_h_enc]
Bi_GRU_encoder_model = Model(Bi_GRU_encoder_inputs, [Bi_GRU_encoder_outputs] + Bi_GRU_encoder_states)

# Rebuild Decoder
latent_dim = state_h_enc.shape[-1]  # Get the latent dimension size from the concatenated states
GRU_decoder_inputs = model2.input[1]  # input_2
GRU_decoder_state_input_h = keras.Input(shape=(latent_dim ,))
GRU_decoder_states_inputs = [GRU_decoder_state_input_h]
GRU_decoder = model2.layers[4]
GRU_decoder_outputs, state_h_dec = GRU_decoder(
    GRU_decoder_inputs, initial_state=GRU_decoder_states_inputs
)
GRU_decoder_states = [state_h_dec]
GRU_decoder_dense = model2.layers[5]
GRU_decoder_outputs = GRU_decoder_dense(GRU_decoder_outputs)
GRU_decoder_model = keras.Model(
    [GRU_decoder_inputs] + GRU_decoder_states_inputs, [GRU_decoder_outputs] + GRU_decoder_states
)

# Testing the rebuilt models (Optional)
print("Encoder model summary:")
Bi_GRU_encoder_model.summary()
print("Decoder model summary:")
GRU_decoder_model.summary()

['input_3', 'bidirectional_1', 'input_4', 'concatenate_2', 'gru_1', 'dense_1']
Encoder model summary:
Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None, 70)]           0         []                            
                                                                                                  
 bidirectional_1 (Bidirecti  [(None, 512),                503808    ['input_3[0][0]']             
 onal)                        (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                                  
 concatenate_13 (Concatenat  (None, 512)                  0         ['bidirectional_1[0]

In [25]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence2(input_seq):
    # Encode the input sequence to get initial states for decoder
    encoder_output, state_h_enc = Bi_GRU_encoder_model.predict(input_seq, verbose=0)
    states_value = [state_h_enc]
    '''encoder_output, state_h_enc_forward, state_h_enc_backward = Bi_GRU_encoder_model.predict(input_seq, verbose=0)
    states_value = [keras.layers.Concatenate()([state_h_enc_forward, state_h_enc_backward])]'''

    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Initialize an empty decoded sentence
    decoded_sentence = ""
    #decoded_sentences=[]


    # Sampling loop for decoding
    stop_condition = False
    while not stop_condition:
        # Predict the next token and update states
        #output_tokens, h = GRU_decoder_model.predict([target_seq, states_value], verbose=0)

        output_tokens, h = GRU_decoder_model.predict(
          [target_seq] + states_value, verbose=0
        )


        # Sample a token from the output distribution
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence for the next time step
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h]

    return decoded_sentence
decoded_sentences=[]
smooth = SmoothingFunction().method4
bleu_score=0
for seq_index in range(20):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences.append(decoded_sentence)
    print(f'Input text: {input_test[seq_index]}')
    print(f'Target text: {target_test[seq_index]}')
    print(f'Decoded text: {decoded_sentence}')
    reference = target_test[seq_index].strip().split()  # Split target text into tokens
    candidate = decoded_sentence.split()  # Split decoded text into tokens
    bleu_score += sentence_bleu(reference, candidate, smoothing_function=smooth)  # Calculate BLEU score

print("Overall BLEU Score: " + str(bleu_score/20))



Input text: Stop yelling!
Target text: 	Cessez de hurler !

Decoded text: Arrêtez de vous dispuser !

Input text: Anybody hurt?
Target text: 	Quiconque a-t-il été blessé ?

Decoded text: Y a-t-il que qu  suis ?

Input text: All aboard!
Target text: 	Tout le monde à bord !

Decoded text: Trapez calme-toi.

Input text: Are you lame?
Target text: 	Êtes-vous boiteux ?

Decoded text: Est-ce que tu au ?

Input text: Who are you?
Target text: 	T'es qui toi ?

Decoded text: Qui s'est échappé ?

Input text: They came in.
Target text: 	Ils sont entrés.

Decoded text: Elles ont vient.

Input text: Get lost.
Target text: 	Décampe !

Decoded text: Dégage.

Input text: I don't date.
Target text: 	Je ne sors pas avec des garçons.

Decoded text: Je ne sors pas pas.

Input text: They love me.
Target text: 	Elles m'aiment.

Decoded text: Ils m'aiment.

Input text: Sign up.
Target text: 	Inscris-toi.

Decoded text: Dégarrez-le.

Input text: We're home.
Target text: 	Nous sommes chez nous.

Decoded text: 

Inference Model and Decoding Sequence for DOuble Layer GRU Encoder and GRU decoder

In [5]:
from keras.models import load_model

# Load the model
model3 = load_model('/content/drive/MyDrive/Colab Notebooks/doubleLayer_gru_Encoder_gru_decoderV2.keras')
for layer in model3.layers:
    print(layer.name)
encoder_temp=model3.input[0]
out,h1=model3.layers[3].output
e_states=[h1]
encoder_model=Model(encoder_temp,e_states)

#decoder_input = Input(shape=(None, 256))
decoder_input=model3.input[1]
#decoder_state_input_h=Input(shape=(512,))
decoder_state_input_h=Input(shape=(latent_dim,))

decoder_states_input =[decoder_state_input_h]
decoder_gru=model3.layers[4]
decoder_outputs, state_h_dec = decoder_gru(decoder_input, initial_state=decoder_states_input)
decoder_dense = model3.layers[-1]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_input] + [decoder_states_input],
    [decoder_outputs, state_h_dec]
)






input_6
gru_3
input_7
gru_4
gru_5
dense_2


In [8]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input sequence to get initial states for decoder
    state_h_enc =encoder_model.predict(input_seq, verbose=0)
    states_value = [state_h_enc]


    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Initialize an empty decoded sentence
    decoded_sentence = ""
    #decoded_sentences=[]


    # Sampling loop for decoding
    stop_condition = False
    while not stop_condition:
        # Predict the next token and update states
        #output_tokens, h = GRU_decoder_model.predict([target_seq, states_value], verbose=0)

        output_tokens, h = decoder_model.predict(
          [target_seq] + states_value, verbose=0
        )


        # Sample a token from the output distribution
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence for the next time step
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h]

    return decoded_sentence
decoded_sentences=[]
smooth = SmoothingFunction().method4
bleu_score=0
for seq_index in range(20):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentences.append(decoded_sentence)
    print('Input text:',input_test[seq_index])
    print('Target text:',target_test[seq_index].strip())
    print('decoded text',decoded_sentence)
    reference = target_test[seq_index].strip().split()  # Split target text into tokens
    candidate = decoded_sentence.split()  # Split decoded text into tokens

    bleu_score += sentence_bleu(reference, candidate, smoothing_function=smooth)  # Calculate BLEU score

print("Overall BLEU Score: " + str(bleu_score/20))





Input text: Stop yelling!
Target text: Cessez de hurler !
decoded text Arrêtez de viel !

Input text: Anybody hurt?
Target text: Quiconque a-t-il été blessé ?
decoded text Y a-t-il que c'est ?

Input text: All aboard!
Target text: Tout le monde à bord !
decoded text Bonne un !

Input text: Are you lame?
Target text: Êtes-vous boiteux ?
decoded text Es-tu fou ?

Input text: Who are you?
Target text: T'es qui toi ?
decoded text Qui est-il ?

Input text: They came in.
Target text: Ils sont entrés.
decoded text Ils ont rieux.

Input text: Get lost.
Target text: Décampe !
decoded text Décampez !

Input text: I don't date.
Target text: Je ne sors pas avec des garçons.
decoded text Je ne peux pas vetir.

Input text: They love me.
Target text: Elles m'aiment.
decoded text Ils a pas le main.

Input text: Sign up.
Target text: Inscris-toi.
decoded text Pas un génie.

Input text: We're home.
Target text: Nous sommes chez nous.
decoded text Nous sommes seuls.

Input text: Have some ham.
Target tex